In [1]:
import keras
from keras import layers
from keras.layers.core import Dense, Activation
from keras.models import Sequential
from keras.callbacks import EarlyStopping
from keras.callbacks import ModelCheckpoint

import pandas as pd

from sklearn.model_selection import train_test_split

import numpy as np

from sklearn.metrics import accuracy_score

from keras.layers import LSTM
from sklearn import preprocessing

from sklearn.ensemble import RandomForestClassifier

Using TensorFlow backend.


In [2]:
# Convert a Pandas dataframe to the x,y inputs that TensorFlow needs
def to_xy(df, target):
    result = []
    for x in df.columns:
        if x != target:
            result.append(x)
    # find out the type of the target column.  Is it really this hard? :(
    target_type = df[target].dtypes
    target_type = target_type[0] if hasattr(target_type, '__iter__') else target_type
    # Encode to int for classification, float otherwise. TensorFlow likes 32 bits.
    if target_type in (np.int64, np.int32):
        # Classification
        dummies = pd.get_dummies(df[target])
        return df.as_matrix(result).astype(np.float32), dummies.as_matrix().astype(np.float32)
    else:
        # Regression
        return df.as_matrix(result).astype(np.float32), df.as_matrix([target]).astype(np.float32)
    
# Encode text values to indexes(i.e. [1],[2],[3] for red,green,blue).
def encode_text_index(df, name):
    le = preprocessing.LabelEncoder()
    df[name] = le.fit_transform(df[name])
    return le.classes_

In [4]:
df = pd.read_csv('data/BTC-USD-edited.csv')
df.head()

,Date,Value USD,Max 7,Min 7,Mean 7,Change,Mean Change 7,Drop 7,Up 7,Predict
0,2010-07-22,0.06262,0.08584,0.04951,0.069031,0.00808,-0.000719,3.0,4.0,-1.0
1,2010-07-23,0.05454,0.08584,0.05050,0.069750,0.00404,0.005049,2.0,5.0,2.0
2,2010-07-24,0.05050,0.08080,0.05050,0.064701,-0.00550,0.003543,3.0,4.0,-1.0
3,2010-07-25,0.05600,0.07921,0.05050,0.061159,-0.00400,0.002106,4.0,3.0,-2.0
4,2010-07-26,0.06000,0.07921,0.05050,0.059053,0.00110,0.002901,3.0,4.0,2.0


In [7]:
df2 = pd.read_csv('data/EOD-MSFT-edited.csv')

In [8]:
df2['Actual'] = df2.apply(actual_calc, axis=1)
df2.head()

KeyError: ('USD 2', 'occurred at index 0')

In [ ]:
X = df[['Value USD', 'Max 7', 'Min 7', 'Change', 'Mean Change 7', 'Drop 7', 'Up 7']].values
y = df['Actual'].values

In [ ]:
df2['USD 2'] = df2['Value USD'].shift(-1)
df2.dropna(inplace=True)
df2.head()

In [ ]:
df2['Actual'] = df2.apply(actual_calc, axis=1)
df2.head()

In [ ]:
df['USD 2'] = df['Actual']
df.head()

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.20, random_state=42)

In [ ]:
clf = RandomForestClassifier(max_depth=2, random_state=0)
clf.fit(X_train, y_train)

In [ ]:
clf.score(X_test, y_test)

In [ ]:
from sklearn.externals import joblib

In [ ]:
joblib.dump(clf, 'data/bitcoin-predictor.pkl')

In [ ]:
clf = joblib.load('data/bitcoin-predictor.pkl')

In [ ]:
[0.05050, 0.08584, 0.04951, -0.01212, -0.001873, 3.0, 3.0]

In [ ]:
clf.predict(np.array([0.05050, 0.08584, 0.04951, -0.01212, -0.001873, 3.0, 3.0]).reshape(1, -1))[0]

In [ ]:
clf.score(X_test, y_test)

In [ ]:
df.iloc[0]

In [ ]:
X2 = df2[['Value USD', 'Max 7', 'Min 7', 'Change', 'Mean Change 7', 'Drop 7', 'Up 7']].values
y2 = df2['Actual'].values

In [ ]:
clf.score(X2, y2)

In [ ]:
df.drop(['Date'], inplace=True, axis=1)
df.head()

In [ ]:
X,y = to_xy(df,"Actual")

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.20, random_state=42)

In [ ]:
model = Sequential()
model.add(Dense(80, activation='relu', input_dim=X_train.shape[1]))
model.add(Dense(80, activation='relu'))
model.add(Dense(80, activation='relu'))
model.add(Dense(80, activation='relu'))
model.add(Dense(80, activation='relu'))
model.add(Dense(y_train.shape[1],activation='softmax'))

#model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])
#model.fit(X_train, y_train, epochs=1000)

model.compile(loss='categorical_crossentropy', optimizer='adam')
monitor = EarlyStopping(monitor='val_loss', min_delta=1e-2, patience=25, verbose=1, mode='auto')
checkpointer = ModelCheckpoint(filepath="best_weights.hdf5", verbose=0, save_best_only=True) # save best model

model.fit(X_train, y_train, validation_data=(X_test, y_test), callbacks=[monitor,checkpointer], verbose=2, epochs=1000)
model.load_weights('best_weights.hdf5') # load weights from best model

In [ ]:
model.save('data/amzn-predictor-shuffled.hd5')

In [ ]:
pred = model.predict(X_test)

In [ ]:
print("Shape: {}".format(pred.shape))
print(pred)

In [ ]:
predict_classes = np.argmax(pred,axis=1)
print("Predictions: {}".format(predict_classes))
print("Expected: {}".format(np.argmax(y_test, axis=1)))
y_test_arg = np.argmax(y_test, axis=1)

In [ ]:
correct = accuracy_score(y_test_arg,predict_classes)
print("Accuracy: {}".format(correct))

In [ ]:
df2 = pd.read_csv('data/AAPL-edited.csv')
df2.head()

In [ ]:
X,y = to_xy(df2,"Actual")

In [ ]:
pred = model.predict(X)

In [ ]:
print("Shape: {}".format(pred.shape))
print(pred)

In [ ]:
predict_classes = np.argmax(pred,axis=1)
print("Predictions: {}".format(predict_classes))
print("Expected: {}".format(np.argmax(y, axis=1)))
y_arg = np.argmax(y, axis=1)

In [ ]:
correct = accuracy_score(y_arg,predict_classes)
print("Accuracy: {}".format(correct))

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.20)

In [ ]:
df3 = pd.read_csv('data/AMZN-edited2.csv')
df3.head()

In [ ]:
df3.drop(['Date', 'High', 'Low', 'Close', 'Volume', 'Open 2', 'Predict', 'Prediction'], axis=1, inplace=True)
df3.head()

In [ ]:
df3.drop('Adj Close', axis=1, inplace=True)
df3.head()

In [ ]:
X,y = to_xy(df3,"Actual")

In [ ]:
pred = model.predict(X)

In [ ]:
print("Shape: {}".format(pred.shape))
print(pred)

In [ ]:
predict_classes = np.argmax(pred,axis=1)
print("Predictions: {}".format(predict_classes))
print("Expected: {}".format(np.argmax(y, axis=1)))
y_arg = np.argmax(y, axis=1)

In [ ]:
correct = accuracy_score(y_arg,predict_classes)
print("Accuracy: {}".format(correct))

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.20)

In [ ]:
pred = model.predict(X_test)

In [ ]:
print("Shape: {}".format(pred.shape))
print(pred)

In [ ]:
predict_classes = np.argmax(pred,axis=1)
print("Predictions: {}".format(predict_classes))
print("Expected: {}".format(np.argmax(y_test, axis=1)))
y_arg = np.argmax(y_test, axis=1)

In [ ]:
correct = accuracy_score(y_arg,predict_classes)
print("Accuracy: {}".format(correct))